# Tox 21

The main goal of Tox21 is to develop better toxicity assessment methods that use high-throughput screening and computational methods to replace traditional animal testing.

In the assay they use thousands of chemicals to test their interaction with a wide range of biological targets using high-throughput assays. Focuses on understanding how chemicals affect biological pathways relevant to disease or adverse outcomes.

→ Common use of the dataset (Task):  
- Focus on **understanding chemical toxicity** at a systemic level, including **effects on various biological pathways**.
- Data is often used for risk assessment and regulatory decision-making.
- We will focus on the toxocity pathwa : NR signaling , by retrieving assay readout 

## The files

- *Tox21_assay_list.xltx* : Exhaustive list of assay conducted on NR. A total of 38 assay are conducted on NR 
- *tox21_10k_library_info.xls* : List of compouds tested in the assay with their CID and CAS identifier, unified identifier for 
- *Tox21Assay_SLPs and Descriptoons_2016* : 38 assay compressed, each folder have 4 files, we are interested in concatenating the __.aggregated.txt__ files into one csv file
    - The code to concatenate the assay was previously done therefore we will load the result saved under *Tox21_assay_aggregated.csv*

- Known target toward NR ; AR, ERα, FXR, PPARδ, PPARγ, RXR, TRβ, VDR, GR, hPXR, AhR, rPXR, CAR, ERR

## Import Libraries

In [1]:
import re
import sys
import warnings
import glob , os
import pandas as pd

In [2]:

def create_directory(path):
    """
    Checks if a directory exists at the given path. If it doesn't, the directory is created.
    
    Args:
        path (str): The path of the directory to check and create.
    """
    if not os.path.exists(path):
        os.mkdir(path)
        print(f"Directory '{path}' created.")
    else:
        print(f"Directory '{path}' already exists.")


## Load the files

In [ ]:
assay_list = pd.read_excel('../Data/Annotations/Tox21_assay_list.xltx')
assay_list.head()

In [ ]:
assay_list_nr = assay_list[assay_list['Target Category'] == 'NR']
print(f'There are {len(assay_list_nr)} NR different assays in the Tox21 dataset')
assay_list_nr

In [ ]:
receptor =  assay_list_nr['Protocol Name'].str.replace(r'[\xa0]','', regex=True).str.split('-').str[1]
receptor = list(set(receptor))
print(f'There are {len(receptor)} different NR receptors tested in the Tox21 dataset')

We will only retain the protocols conducted under the same assay readout, specifically the BLA (b-lactamase) reporter gene assay. Different assay readouts are as follows:

- **Luminescence**: e.g., AR-MDA_TOX21_SLP_Version1.0 (non-receptor signaling)
- **Luciferase Reporter**: e.g., tox21-ahr-p1 (Aryl Hydrocarbon Receptor (AhR)); CAR1_TOX21_SLP_Version1.0

Please note that the assay version **Version1.0** is indicated as **-p1** in the assay name.

In [ ]:
blactamase_assay = []

for assay in assay_list_nr['Protocol Name']:

    if assay.split('-')[2] == 'bla' or (len(assay.split('-')) > 3 and assay.split('-')[3] == 'bla'):
        blactamase_assay.append(assay)

unique_receptors = list(set(item.split('-')[1] for item in blactamase_assay))

# Print the results
print(f"There are {len(blactamase_assay)} BLA assays for {len(unique_receptors)} unique receptors, namely {unique_receptors}.")

In [7]:
blactamase_assay = [assay.replace('\xa0', '') for assay in blactamase_assay]

We load the library of 10,000 compounds and the Tox21Assay_SLPS assay from 2016, available for download [here](https://clowder.edap-cluster.com/datasets/63602c6de4b04f6bb13dc4d4)

In [ ]:
''' 
path = ('../Data_reviewed/Tox21/Assay_aggregrated')

all_assay = glob.glob(os.path.join(path, "*.txt"))
non_empty_assays = [pd.read_table(f) for f in all_assay if not pd.read_table(f).empty]
Tox21_assay = pd.concat(non_empty_assays, ignore_index=True)

Tox21_assay.to_csv('../Data_reviewed/Tox21/Tox21_assay_aggregrated.csv', index=False)
print(f"There is {Tox21_assay['CAS'].nunique()} unique compounds in the Tox21 aggegated dataset")
'''

In [ ]:
Tox21_assay = pd.read_csv('../Data/Annotations/Tox21_assay_aggregrated.csv',low_memory=False)
print(f"There is {Tox21_assay['CAS'].nunique()} unique compounds in the Tox21 aggegated dataset")
Tox21_assay.head(3)

In [ ]:
library_info = pd.read_csv('../Data/Annotations/tox21_10k_library_info.tsv', delimiter='\t')
print(f"There is {library_info['CAS'].nunique()} unique compounds in the Tox21 chemical library dataset")

In [ ]:
print(f"There are {Tox21_assay['PROTOCOL_NAME'].nunique()} different assays retrieved, not all of directed toward  NR assays.")

- We only keep the assay that are relevant for us , ie, assay toward NR ; AR, ERα, FXR, PPARδ, PPARγ, RXR, TRβ, VDR, GR, hPXR, AhR, rPXR, CAR, ERR

In [ ]:
Tox21_assay = Tox21_assay[Tox21_assay['PROTOCOL_NAME'].isin(blactamase_assay)]
print(f"There are {Tox21_assay['PROTOCOL_NAME'].nunique()} BLA readout assays toward NR receptor .")

In [ ]:
sorted_protocol_names = sorted(set(Tox21_assay['PROTOCOL_NAME']))
print(f'There are 15 BLA readout assays toward NR receptor : {sorted_protocol_names}')

In [ ]:
create_directory('../Data/Output')

In [15]:
Tox21_annotation = Tox21_assay.merge( library_info, on='SAMPLE_ID', how='right')
Tox21_annotation.to_csv('../Data/Output/Tox21_annotation.csv',index=False)

Load morphology annotation file : 
- See how many compound overlap in BBC047 and Tox21

In [16]:
#profiles = pd.read_table('../pubchem_annotation_morpho.csv')
#profiles.rename(columns={'CID': 'PUBCHEM_CID'}, inplace=True)
#print(f"There are {profiles['PUBCHEM_CID'].nunique()} unique compounds in the profiles dataset")
#print(f'There are {profiles["PUBCHEM_CID"].isin(Tox21_annotation["PUBCHEM_CID"]).sum()} compounds in the BBC047 dataset that are also in the Tox21 library')
#profiles.head(2)

We used PubChem to retrieve the CID for the 30,615 unique substances tested in BBC047. Since CPD_NAME can identify several molecules without considering stereochemistry, we used Metadata_broad_sample to query PubChem for the CID. If this was unsuccessful, we used the SMILES representation. Five molecules were not identified, resulting in 30,397 unique substances. The CID retrieval code in get_assays was last run on 27/10/2024, taking approximately 4 hours.
the outpout is saved under _pubchem_annotation_october_2.pkl_


In [17]:
annotations_update = pd.read_pickle('../Data/Annotations/pubchem_annotation1.pkl') #pubchem_annotation1_october_2

In [ ]:
annotations_update.rename(columns={'CID': 'PUBCHEM_CID'}, inplace=True)
annotations_update.head(5)

In [ ]:
print(f"There are {annotations_update['PUBCHEM_CID'].nunique()} unique compounds in the unprocessed profiles dataset")

In [ ]:
print(f'There are {annotations_update["PUBCHEM_CID"].isin(Tox21_annotation["PUBCHEM_CID"]).sum()} compounds in the BBC047 dataset that are also in the Tox21 library')

Now we merge the morphology file and the tox21

In [21]:
BBC047_mol = annotations_update['PUBCHEM_CID'].to_list() # profiles['PUBCHEM_CID'].to_list()
Tox = Tox21_annotation[Tox21_annotation['PUBCHEM_CID'].isin(BBC047_mol)]

In [ ]:
print(f"There are {Tox['SAMPLE_DATA_TYPE'].nunique()} sample data types, namely {list(Tox['SAMPLE_DATA_TYPE'].unique())}")

We are only interested in the activity

In [23]:
Tox_activity = Tox[Tox['SAMPLE_DATA_TYPE'] == 'activity']

In [ ]:
print(f"There are {Tox_activity['SAMPLE_NAME_x'].nunique()} unique sample molecules in the BBC047 dataset tested in Tox21 assays")

In [ ]:
create_directory('../Data/Output')
Tox_activity.to_csv('../Data/Output/Tox21_activity_BBC047.csv',index=False)

## Dataset curation ; some descriptive analysis

In [26]:
import plotly.express as px

We first start by counting the number of compound (unique) per assay (unique)

In [27]:
assay_compound = {}
for asssay in Tox_activity['PROTOCOL_NAME'].unique():
    assay_compound[asssay] = Tox_activity[Tox_activity['PROTOCOL_NAME'] == asssay].CAS_x.nunique()

In [ ]:
assay_compound

In [29]:
assay_compound_df = pd.DataFrame(list(assay_compound.items()), columns=['Assay', 'Compound'])
assay_compound_df = assay_compound_df.sort_values(by='Compound', ascending=True)

In [ ]:
fig = px.bar(assay_compound_df, x='Assay', y='Compound', title='Number of compounds tested in each assay')
fig.show()

-  look at all combinations of how sets intersect : using the upsetplot and library we will show the overlap of compound between the assays
** to update **

In [31]:
import upsetplot
from collections import defaultdict
import matplotlib.pyplot as plt
from upsetplot import UpSet
from upsetplot import UpSet, from_memberships

In [32]:
assay_data = {}
for asssay in blactamase_assay:
    assay_data[asssay] = Tox_activity[Tox_activity['PROTOCOL_NAME'] == asssay].CAS_x.unique().tolist()

In [33]:
#TO BE CONTINUED ...

## Endocrine active molecule 

We aim to create a dataset of endocrine-active molecules. Each compound as tested twice for each assay. If the assay outcomes are inconsistent, we will exclude the assay for that compound. Additionally, if an assay is inconclusive—meaning the experimenters could not determine whether the molecule was an agonist, antagonist, inactive, or active—the assay will be discarded. Finally, we will evaluate the remaining assays: if at least one is active, the molecule will be classified as endocrine-active; if all assays are inactive, the compound will be classified as inactive.

We 1st consider the assay measuring the *activty*, therefore we will focus on the *channel outcome* column., reflecting the outcome of the assay under investigation. 

Note : If an assay measures two channels (e.g., a receptor activity channel and a cell viability channel), the CHANNEL_OUTCOME might be “Active” for the receptor but “Inactive” for cell viability. The overall ASSAY_OUTCOME could then be “Inconclusive” or “Active” depending on how the results are interpreted.

In [ ]:
Tox_activity.head(2)

In [35]:
molecule_list = list(Tox_activity['PUBCHEM_CID'].unique()) #SAMPLE_NAME_x
Endocrine_activity = Tox_activity.copy(deep=True)
assay_list_endocrine = list(Endocrine_activity['PROTOCOL_NAME'].unique())

In [36]:
activity_dict = {}

In [37]:
def define_endocrine_activity(mol_list, assay_list, data, activity_dict):
    """
    Process list of molecules overlapping with BBC047 by checking their assay outcomes and define their endocrine activity status.
    
    Parameters:
    mol_list (list): List of molecules to process.
    assay_list (list): List of assays to check for each molecule.
    data (DataFrame): DataFrame containing assay data for different molecules.
    activity_dict (dict): Dictionary to store activity status ('active' or 'inactive') for each molecule.
    
    Returns:
    DataFrame: Updated data_test after filtering out rows based on conditions.
    dict: Updated activity_dict with activity status of molecules.
    """
    #check 
    for mol in mol_list:
        view_mol = data[data['PUBCHEM_CID'] == mol].copy() #'SAMPLE_NAME_y'

        for assay in assay_list:
            view_assay = view_mol[view_mol['PROTOCOL_NAME'] == assay].copy()


            if len(set(view_assay['CHANNEL_OUTCOME'])) > 1:
                data = data[~data.index.isin(view_assay.index)]

            if any(re.search(r'\binconclusive\b', outcome) for outcome in view_assay['CHANNEL_OUTCOME']):
                data = data[~data.index.isin(view_assay.index)]
                continue
        if any(re.search(r'\bactive\b', outcome) for outcome in set(view_mol['CHANNEL_OUTCOME'])):
            activity_dict[mol] = 'active'
        else:
            activity_dict[mol] = 'inactive'

    return data, activity_dict


In [38]:
updated_data , activity = define_endocrine_activity(molecule_list, assay_list_endocrine, Endocrine_activity, activity_dict)

In [39]:
# Compter les occurrences de 'active' et 'inactive'
active_count = sum(1 for value in activity_dict.values() if value == 'active')
inactive_count = sum(1 for value in activity_dict.values() if value == 'inactive')

In [ ]:
print(f'Number of active molecules: {active_count}')
print(f'Number of inactive molecules: {inactive_count}')

_Note_: This activity is defined in relation to all NR (nuclear receptor) tested in Tox21, which are classified as endocrine active molecules. We do not consider GPCRs (G protein-coupled receptors) as a pathway for endocrine activity for this case.

Therefore, if a molecule is classified as active in at least one assay after removing inconsistent and inconclusive results, it will be annotated as active. For example, p,p’-DDT is inactive in 8 out of the 9 remaining assays; however, it is active in all 3 replicates of the *Tox21 ER-Bla Agonist P2* assay and will be annotated as endocrine active.



In [ ]:
view_mol = updated_data[updated_data['SAMPLE_NAME_x'] == "p,p'-DDT"].copy() #CAS_x ; 50-29-3 
# Trier le DataFrame view_mol en fonction de la colonne 'PROTOCOL_NAME'

view_mol.sort_values(by='PROTOCOL_NAME')

In [ ]:
view_mol['PROTOCOL_NAME'].unique()

## Save the data 

He save as a csv file the data we need to merge to the BBC047 dataset. That is the activity status, the name of the molecule, the smiles and CID.


In [ ]:
endocrine_annotation_cleaned = pd.DataFrame(list(activity.items()), columns=['PUBCHEM_CID', 'Endocrine_activity']) #CPD_NAME
Endocrine_activity['PUBCHEM_CID'] = Endocrine_activity['PUBCHEM_CID'].astype(int)
endocrine_annotation_cleaned.head(2)


In [ ]:
Endocrine_activity.head(3)

In [45]:
Endocrine_activity = Endocrine_activity[['CAS_x','SMILES','PUBCHEM_CID','SAMPLE_NAME_x']]
Endocrine_activity.rename(columns={'SAMPLE_NAME_x':'CPD_NAME', 'CAS_x':'CAS'}, inplace=True)

In [46]:
Endocrine_activity = endocrine_annotation_cleaned.merge(Endocrine_activity, on='PUBCHEM_CID', how='right')

In [ ]:
endocrine_annotation_cleaned

In [48]:
Endocrine_activity['PUBCHEM_CID'] = Endocrine_activity['PUBCHEM_CID'].astype(int)
Endocrine_activity = Endocrine_activity.groupby('PUBCHEM_CID').first().reset_index()

In [ ]:
Endocrine_activity.head(2)

In [ ]:
print(f"{Endocrine_activity['PUBCHEM_CID'].nunique()} unique compounds in the endocrine activity dataset")
Endocrine_activity.head(3)

In [ ]:
create_directory('../Data/Output')
Endocrine_activity.to_csv('../Data/Output/Tox21_Endocrine_activity_BBC047.csv',index=False)

We retrieved the missing cid from the pre-processed , to see which molecules are concerned.

In [52]:
missing_cid = [2724385,
 2082,
 5154,
 26596,
 6758,
 3334,
 439501,
 10607,
 36314,
 30323,
 6197,
 15478,
 6167,
 4122,
 4030]

In [ ]:
Endocrine_activity[Endocrine_activity['PUBCHEM_CID'].isin(missing_cid)]

In [ ]:
annotations_update[annotations_update['PUBCHEM_CID'].isin(missing_cid)]

## Generate a column for plot in Datagrok 

In [55]:
T = [
    'Bisphenol A diglycidyl ether', 'Dexamethasone acetate', 'Testosterone propionate',
    'Estrone', "p,p'-DDT"
]

In [56]:
Endocrine_activity['Datagrok'] = Endocrine_activity['CPD_NAME'].apply(lambda x: x if x in T else '')

In [ ]:
create_directory('../Data/Output')
Endocrine_activity.to_csv('../Data/Output/Tox21_Endocrine_Datagrok.csv',index=False)